The purpose of this notebook is to define and persist the model to be used by the QA Bot accelerator.  This notebook is available at https://github.com/databricks-industry-solutions/diy-llm-qa-bot.

##Introduction

With our documents indexed, we can now focus our attention on assembling the core application logic.  This logic will have us retrieve a document from our vector store based on a user-provided question.  That question along with the document, added to provide context, will then be used to assemble a prompt which will then be sent to a model in order to generate a response. </p>

<img src='https://brysmiwasb.blob.core.windows.net/demos/images/bot_application.png' width=900>

</p>
In this notebook, we'll first walk through these steps one at a time so that we can wrap our head around what all is taking place.  We will then repackage the logic as a class object which will allow us to more easily encapsulate our work.  We will persist that object as a model within MLflow which will assist us in deploying the model in the last notebook associated with this accelerator.

In [0]:
%pip install langchain==0.0.250 tiktoken==0.4.0 openai==0.27.6 faiss-cpu==1.7.4 typing-inspect==0.8.0 typing_extensions==4.5.0

Python interpreter will be restarted.
  Using cached langchain-0.0.250-py3-none-any.whl (1.4 MB)
  Using cached tiktoken-0.4.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
  Using cached openai-0.27.6-py3-none-any.whl (71 kB)
  Using cached faiss_cpu-1.7.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
  Using cached typing_inspect-0.8.0-py3-none-any.whl (8.7 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Using cached openapi_schema_pydantic-1.2.4-py3-none-any.whl (90 kB)
  Using cached langsmith-0.0.21-py3-none-any.whl (32 kB)
  Using cached numexpr-2.8.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (382 kB)
  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)
  Using cached SQLAlchemy-2.0.19-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.8 MB)
  Using cached dataclasses_json-0.5.14-py3-none-any.whl (26 kB)
  Using cached aiohttp-3.8.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.

In [0]:
import re
import time
import pandas as pd
import mlflow
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.schema import BaseRetriever
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.prompts.base import BasePromptTemplate
from langchain.prompts import PromptTemplate
from langchain.base_language import BaseLanguageModel
from langchain import LLMChain

In [0]:
%run "./utils/config_utils"

In [0]:
config

Out[10]: {'kb_documents_path': 'file:/Workspace/Users/aurelia.ayala@factored.ai/ICBF_QABot/data/icbf_knowledge_base.csv',
 'vector_store_path': '/dbfs/tmp/icbf_qabot/vector_store',
 'database_name': 'icbf_qabot',
 'registered_model_name': 'databricks_icbf_qabot',
 'model_uri': 'models:/databricks_icbf_qabot/production',
 'openai_embedding_model': 'text-embedding-ada-002',
 'openai_chat_model': 'gpt-3.5-turbo',
 'system_message_template': "You are a helpful bilingual assistant. You are good at helping to answer a question written in Spanish or English based on the context provided, the context is a document in Spanish. If the context does not provide enough relevant information to determine the answer, just say I don't know. If the context is irrelevant to the question, just say I don't know. If you did not find a good answer from the context, just say I don't know. If the query doesn't form a complete question, just say I don't know. If there is a good answer from the context, try to s

##Step 1: Explore Answer Generation

To get started, let's explore how we will derive an answer in response to a user provide question.  We'll start by defining that question here:

In [0]:
question = "¿Cómo puedo interponer una demanda de alimentos?"

Using our vector store, assembled in the prior notebook, we will retrieve document chunks relevant to the question: 

**NOTE** The OpenAI API key used by the OpenAIEmbeddings object is specified in an environment variable set during the earlier `%run` call to get configuration variables.

In [0]:
# open vector store to access embeddings
embeddings = OpenAIEmbeddings(model=config['openai_embedding_model'])
vector_store = FAISS.load_local(embeddings=embeddings, folder_path=config['vector_store_path'])

# configure document retrieval 
n_documents = 5 # number of documents to retrieve 
retriever = vector_store.as_retriever(search_kwargs={'k': n_documents}) # configure retrieval mechanism

# get relevant documents
docs = retriever.get_relevant_documents(question)
for doc in docs: 
  print(doc,'\n') 

page_content='Luego de presentada la solicitud, las partes implicadas en la fijación de la cuota de alimentos, son citadas a una audiencia de conciliación, con el propósito de que lleguen a un acuerdo sobre el monto de la misma. De no lograr acuerdo alguno, la autoridad que este conociendo del trámite, mediante resolución, fijará provisionalmente las obligaciones de alimentos, custodia y visitas, respecto del niño, niña o adolescente. De llegarse a un acuerdo respecto del monto de la cuota de alimentos, se elaborará acta en la que se exprese dicho acuerdo, el Defensor de Familia deberá entregar una copia a las partes interesadas. En caso de incumplimiento en el pago del monto de la cuota de alimentos, el acta de conciliación servirá de prueba para adelantar una demanda ejecutiva de alimentos. El Defensor de Familia solo dará inicio a la demanda si el interesado manifiesta su intención en que sea presentada y así dar inicio al proceso correspondiente. Ahora bien, establecida la cuota al

We can now turn our attention to the prompt that we will send to the model.  This prompt needs to include placeholders for the *question* the user will submit and the document that we believe will provide the *context* for answering it.

Please note that the prompt consists of multiple prompt elements, defined using [prompt templates](https://python.langchain.com/en/latest/modules/prompts/chat_prompt_template.html).  In a nutshell, prompt templates allow us to define the basic structure of a prompt and more easily substitute variable data into them to trigger a response.  The system message prompt shown here provides instruction to the model about how we want it to respond.  The human message template provides the details about the user-initiated request.

The prompts along with the details about the model that will respond to the prompt are encapsulated within an [LLMChain object](https://python.langchain.com/en/latest/modules/chains/generic/llm_chain.html).  This object simply defines the basic structure for resolving a query and returning a response:

In [0]:
# define system-level instructions
system_message_prompt = SystemMessagePromptTemplate.from_template(config['system_message_template'])

# define human-driven instructions
human_message_prompt = HumanMessagePromptTemplate.from_template(config['human_message_template'])

# combine instructions into a single prompt
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# define model to respond to prompt
llm = ChatOpenAI(model_name=config['openai_chat_model'], temperature=config['temperature'])

# combine prompt and model into a unit of work (chain)
qa_chain = LLMChain(
  llm = llm,
  prompt = chat_prompt
  )

To actually trigger a response, we will loop through each of our docs from highest to lowest relevance and attempt to elicit a response.  Once we get a valid response, we'll stop.

Please note, we aren't providing time-out handling or thoroughly validating the response from the model in this next cell.  We will want to make this logic more robust as we assemble our application class but for now we'll keep it simple to ensure the code is easy to read:

In [0]:
# for each provided document
for doc in docs:

  # get document text
  text = doc.page_content

  # generate a response
  output = qa_chain.generate([{'context': text, 'question': question}])
 
  # get answer from results
  generation = output.generations[0][0]
  answer = generation.text

  # display answer
  if answer is not None:
    print(f"Question: {question}", '\n', f"Answer: {answer}")
    break

Question: ¿Cómo puedo interponer una demanda de alimentos? 
 Answer: Para interponer una demanda de alimentos, primero debes presentar una solicitud ante la autoridad competente. Luego, las partes implicadas en la fijación de la cuota de alimentos serán citadas a una audiencia de conciliación para intentar llegar a un acuerdo sobre el monto de la cuota. Si no se logra un acuerdo, la autoridad competente fijará provisionalmente las obligaciones de alimentos. En caso de incumplimiento en el pago de la cuota de alimentos, el acta de conciliación servirá como prueba para presentar una demanda ejecutiva de alimentos. Sin embargo, es importante mencionar que el Defensor de Familia solo iniciará la demanda si el interesado manifiesta su intención de presentarla. Después de establecer la cuota alimentaria provisional, el interesado deberá acudir a la jurisdicción de familia para iniciar el proceso correspondiente de fijación de la cuota alimentaria.


##Step 2: Assemble Model for Deployment

Having explored the basic steps involved in generating a response, let's wrap our logic in a class to make deployment easier.  Our class will be initialized by passing the LLM model definition, a vector store retriever and a prompt to the class.  The *get_answer* method will serve as the primary method for submitting a question and getting a response:

In [0]:
class QABot():


  def __init__(self, llm, retriever, prompt):
    self.llm = llm
    self.retriever = retriever
    self.prompt = prompt
    self.qa_chain = LLMChain(llm = self.llm, prompt=prompt)
    self.abbreviations = { # known abbreviations we want to replace
      "ICBF": "Instituto Colombiano de Bienestar Familiar",
      } 


  def _is_good_answer(self, answer):

    ''' check if answer is a valid '''

    result = True # default response

    badanswer_phrases = [ # phrases that indicate model produced non-answer
      "no information", "no context", "don't know", "no clear answer", "sorry", 
      "no answer", "no mention", "reminder", "context does not provide", "no helpful answer", 
      "given context", "no helpful", "no relevant", "no question", "not clear",
      "don't have enough information", " does not have the relevant information", "does not seem to be directly related",
      "no tengo información", "no tengo contexto", "no sé", "no hay una respuesta clara", "disculpa", "sin respuesta",
      "no se menciona", "el contexto no proporciona", "no hay una respuesta útil", "sin relevancia", "no hay pregunta",
      "no es claro", "no tengo suficiente información", "no existe información relevante", "no parece estar directamente relacionado"
      ]
    
    if answer is None: # bad answer if answer is none
      results = False
    else: # bad answer if contains badanswer phrase
      for phrase in badanswer_phrases:
        if phrase in answer.lower():
          result = False
          break
    
    return result


  def _get_answer(self, context, question, timeout_sec=60):

    '''' get answer from llm with timeout handling '''

    # default result
    result = None

    # define end time
    end_time = time.time() + timeout_sec

    # try timeout
    while time.time() < end_time:

      # attempt to get a response
      try: 
        result =  qa_chain.generate([{'context': context, 'question': question}])
        break # if successful response, stop looping

      # if rate limit error...
      except openai.error.RateLimitError as rate_limit_error:
        if time.time() < end_time: # if time permits, sleep
          time.sleep(2)
          continue
        else: # otherwise, raiser the exception
          raise rate_limit_error

      # if other error, raise it
      except Exception as e:
        print(f'LLM QA Chain encountered unexpected error: {e}')
        raise e

    return result


  def get_answer(self, question):
    ''' get answer to provided question '''

    # default result
    result = {'answer':None, 'source':None, 'output_metadata':None}

    # remove common abbreviations from question
    for abbreviation, full_text in self.abbreviations.items():
      pattern = re.compile(fr'\b({abbreviation}|{abbreviation.lower()})\b', re.IGNORECASE)
      question = pattern.sub(f"{abbreviation} ({full_text})", question)

    # get relevant documents
    docs = self.retriever.get_relevant_documents(question)

    # for each doc ...
    for doc in docs:

      # get key elements for doc
      text = doc.page_content
      source = doc.metadata['url']

      # get an answer from llm
      output = self._get_answer(text, question)
 
      # get output from results
      generation = output.generations[0][0]
      answer = generation.text
      output_metadata = output.llm_output

      # assemble results if not no_answer
      if self._is_good_answer(answer):
        result['answer'] = answer
        result['source'] = source
        result['output_metadata'] = output_metadata
        break # stop looping if good answer
      
    return result

Now we can test our class using the objects instantiated earlier:

In [0]:
# instantiate bot object
qabot = QABot(llm, retriever, chat_prompt)

# get response to question
qabot.get_answer(question) 

Out[18]: {'answer': 'Para interponer una demanda de alimentos, primero debes presentar una solicitud ante la autoridad competente. Luego, las partes implicadas en la fijación de la cuota de alimentos serán citadas a una audiencia de conciliación para intentar llegar a un acuerdo sobre el monto de la cuota. Si no se logra un acuerdo, la autoridad competente fijará provisionalmente las obligaciones de alimentos. En caso de incumplimiento en el pago de la cuota de alimentos, el acta de conciliación servirá como prueba para presentar una demanda ejecutiva de alimentos. Sin embargo, es importante mencionar que el Defensor de Familia solo iniciará la demanda si el interesado manifiesta su intención de presentarla. Después de establecer la cuota alimentaria provisional, el interesado deberá acudir a la jurisdicción de familia para iniciar el proceso correspondiente de fijación de la cuota alimentaria.',
 'source': 'https://www.icbf.gov.co/en-que-consiste-el-tramite-de-fijacion-de-cuota-de-ali

##Step 3: Persist Model to MLflow

With our bot class defined and validated, we can now persist it to MLflow.  MLflow is an open source repository for model tracking and logging.  It's deployed by default with the Databricks platform, making it easy for us to record models with it.

While MLflow now [supports](https://www.databricks.com/blog/2023/04/18/introducing-mlflow-23-enhanced-native-llm-support-and-new-features.html) both OpenAI and LangChain model flavors, the fact that we've written custom logic for our bot application means that we'll need to make use of the more generic [pyfunc](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#creating-custom-pyfunc-models) model flavor.  This model flavor allows us to write a custom wrapper for our model that gives us considerable control over how our model responds when deployed through standard, MLflow-provided deployment mechanisms. 

To create a custom MLflow model, all we need to do is define a class wrapper of type *mlflow.pyfunc.PythonModel*. The *__init__* method will initialize an instance of our *QABot* class and persist it to an class variable.  And a *predict* method will serve as the standard interface for generating a response.  That method will receive our inputs as a pandas dataframe but we can write the logic with the knowledge that it will only be receiving one user-provided question at a time:

In [0]:
class MLflowQABot(mlflow.pyfunc.PythonModel):

  def __init__(self, llm, retriever, chat_prompt):
    self.qabot = QABot(llm, retriever, chat_prompt)

  def predict(self, context, inputs):
    questions = list(inputs['question'])

    # return answer
    return [self.qabot.get_answer(q) for q in questions]

We can then instantiate our model and log it to the [MLflow registry](https://docs.databricks.com/mlflow/model-registry.html) as follows:

In [0]:
# instantiate mlflow model
model = MLflowQABot(llm, retriever, chat_prompt)

# persist model to mlflow
with mlflow.start_run():
  _ = (
    mlflow.pyfunc.log_model(
      python_model=model,
      extra_pip_requirements=['langchain==0.0.250', 'tiktoken==0.4.0', 'openai==0.27.6', 'faiss-cpu==1.7.4', 'typing-inspect==0.8.0', 'typing_extensions==4.5.0'],
      artifact_path='model',
      registered_model_name=config['registered_model_name']
      )
    )


/databricks/python/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'databricks_icbf_qabot'.
2023/08/11 01:21:59 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: databricks_icbf_qabot, version 1
Created version '1' of model 'databricks_icbf_qabot'.


If you are new to MLflow, you may be wondering what logging is doing for us.  If you navigate to the experiment associated with this notebook - look for the flask icon in the right-hand navigation of your Databricks environment to access the experiments - you can click on the latest experiment to see details about what was recorded with the *log_model* call. If you expand the model artifacts, you should see a *python_model.pkl* file that represents the pickled MLflowQABot model instantiated before.  It's this model that we retrieve when we (later) load our model into this or another environment:
</p>

<img src="https://brysmiwasb.blob.core.windows.net/demos/images/bot_mlflow_log_model.PNG" width=1000>

The MLflow model registry provides mechanisms for us to manage our registered models as they move through a CI/CD workflow.  If we want to just push a model straight to production status (which is fine for a demo but not recommended in real-world scenarios), we can do this programmatically as follows:

In [0]:
# connect to mlflow 
client = mlflow.MlflowClient()

# identify latest model version
latest_version = client.get_latest_versions(config['registered_model_name'], stages=['None'])[0].version

# move model into production
client.transition_model_version_stage(
    name=config['registered_model_name'],
    version=latest_version,
    stage='Production',
    archive_existing_versions=True
)

Out[23]: <ModelVersion: creation_timestamp=1691716919251, current_stage='Production', description='', last_updated_timestamp=1691716930668, name='databricks_icbf_qabot', run_id='2ff6599ab2cd44b3b8f57a1858a3d29b', run_link='', source='dbfs:/databricks/mlflow-tracking/3872731099569187/2ff6599ab2cd44b3b8f57a1858a3d29b/artifacts/model', status='READY', status_message='', tags={}, user_id='7558745341569247', version='1'>

We can then retrieve the model from the registry and submit a few questions to verify the response:

In [0]:
# retrieve model from mlflow
model = mlflow.pyfunc.load_model(f"models:/{config['registered_model_name']}/Production")

# assemble question input
queries = pd.DataFrame({'question':[
  "¿Qué es la patria potestad?",
  "¿Puedo tener la custodia de mi hijo si soy menor de edad?",
  "¿Para qué es una prueba de ADN?"
]})

# get a response
model.predict(queries)

Out[25]: [{'answer': 'La patria potestad es el conjunto de derechos que la ley les reconoce a ambos padres sobre sus hijos menores de edad, incluyendo la administración de sus bienes y representación legal.',
  'source': 'https://www.icbf.gov.co/cual-es-la-diferencia-entre-custodia-y-patria-potestad-0',
  'output_metadata': {'token_usage': {'prompt_tokens': 297,
    'completion_tokens': 44,
    'total_tokens': 341},
   'model_name': 'gpt-3.5-turbo'}},
 {'answer': 'No, según la ley establecida en el contexto, la custodia de los hijos está en primer lugar en cabeza de los padres, incluso si son menores de edad. Sin embargo, si los padres no son aptos para tener la custodia y ofrecer las garantías necesarias para el cuidado del niño, la custodia puede ser otorgada a un familiar.',
  'source': 'https://www.icbf.gov.co/soy-menor-de-edad-puedo-tener-la-custodia-de-mi-hijo',
  'output_metadata': {'token_usage': {'prompt_tokens': 263,
    'completion_tokens': 81,
    'total_tokens': 344},
   '

© 2023 Databricks, Inc. All rights reserved. The source in this notebook is provided subject to the Databricks License. All included or referenced third party libraries are subject to the licenses set forth below.

| library                                | description             | license    | source                                              |
|----------------------------------------|-------------------------|------------|-----------------------------------------------------|
| langchain | Building applications with LLMs through composability | MIT  |   https://pypi.org/project/langchain/ |
| tiktoken | Fast BPE tokeniser for use with OpenAI's models | MIT  |   https://pypi.org/project/tiktoken/ |
| faiss-cpu | Library for efficient similarity search and clustering of dense vectors | MIT  |   https://pypi.org/project/faiss-cpu/ |
| openai | Building applications with LLMs through composability | MIT  |   https://pypi.org/project/openai/ |